# Automate workflow with Step Functions Data Science SDK

## Install SDK

In [ ]:
# !pip install --upgrade stepfunctions

## Setup environment

In [ ]:
import json
import yaml
import logging
import sagemaker
import stepfunctions
from stepfunctions.workflow import Workflow, cloudformation

stepfunctions.set_stream_logger(level=logging.INFO)
role = sagemaker.get_execution_role() # execution role for SageMaker
workflow_execution_role = role # execution role for Step Functions

# get workflow input parameters
with open('workflow_parameters.json') as json_file:
    inputs = json.load(json_file)

## Load steps from step definition file

In [ ]:
# replace first lambda ids in the file with lambda deployed into your account
from step_definitions import *

## Create workflow pipeline

In [ ]:
workflow_pipeline_definition = Chain(
    [
        initialise_execution,
        sagemaker_processing_job,
        wait,
        get_processing_status,
        is_processing_complete
    ]
)

workflow_pipeline = Workflow(
    name="end-to-end-transformers",
    definition=workflow_pipeline_definition,
    role=workflow_execution_role
)

In [ ]:
print(workflow_pipeline.definition.to_json(pretty=True))

In [ ]:
workflow_pipeline.render_graph(portrait=True)

## Create/update state machine and execute

In [ ]:
workflow_pipeline.create()
# workflow_pipeline.update(workflow_pipeline_definition)

In [ ]:
workflow_pipeline.execute(inputs=inputs)

## Generate CloudFormation template

In [ ]:
print(workflow_pipeline.get_cloudformation_template())